# Kahoot Ranking

Este Notebook filtra os vencedores do Kahoot, retornando os primeiros n colocados.

## ▶ Pontuação
### 1. Pontos de pódio
Para cada Kahoot:

🥇1 lugar : 3 pontos

🥈2 lugar : 2 pontos

🥉3 lugar : 1 ponto

### 2. Pontos do Kahoot
O desempate é feito pela pontuação acumulada dos Kahoots. 

In [41]:
import os
import pandas as pd
from unidecode import unidecode
from fuzzywuzzy import process
import re

import logging

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(levelname)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

def clean_string(input_string):
    # Use a regular expression to remove spaces and numbers
    cleaned_string = re.sub(r'[\s\d\W]', '', input_string)

    # Remove accents using unidecode
    string = unidecode(cleaned_string).lower()

    return string

def normalize_name(name, known_names, mapping, threshold=30):
    """
    Normalize a name by checking against a mapping and known names.

    Parameters:
    name (str): The name to normalize.
    known_names (list): A list of known names to compare against.
    mapping (dict): A dictionary mapping names to their normalized forms.
    threshold (int): The minimum score for a match to be considered valid (default is 30).

    Returns:
    str: The normalized name if a match is found; otherwise, returns 'johann'.
    """
    if name in mapping:
        return mapping[name]

    match = process.extractOne(name, known_names)

    if match and match[1] >= threshold:
        return match[0]

    return 'johann'

## Set folder path

In [42]:
# Set folder path
folder_path = "2025-1_METANOIA"

## Name exception dict 
Define name aliases to substitute (check which names have more than 1 alias)

'alias': 'originalname'

In [43]:
name_alias = {
    'mar+bi=marbi': 'mardabi',
    'marsembi': 'mardabi',
    'marbinoso': 'mardabi',
    'marcombi': 'mardabi',
    'marbinado': 'mardabi',
    'marbix': 'mardabi',
    'bine': 'bidomar',
    'binedomar': 'bidomar',
    'sabrine': 'bidomar',
    'mardomar': 'johann',
    'quadra': 'johann',
    'quadrado': 'johann',
    'bigxand': 'xandao',
    'shaquilleomeal': 'johann',
    'bobberkurwa': 'johann',
    'paracetamal': 'johann',
    'quadroh': 'johann',
    'vaixco ': 'johann',
    'bagriel': 'johann',
    'mardomar': 'johann',
    'quadrado': 'johann',
    'luaraa': 'luara',
    'luaraara': 'luara',
    'kakerlake': 'johann',
    'fmr': 'johann',
    'pirarucu': 'johann',
    'tucunare': 'johann',
    'bambu': 'johann',
    'yej!b': 'yejin',
    'yej!n': 'yejin',
    'tirisco': 'johann',
    'luu': 'luara',
    'luuu': 'luara',
    'lua': 'luara',
    'luaura': 'luara',
    'luaraaa': 'luara',
    'gih': 'giovanna',
}

## Main

In [44]:
# Main
# Get file info
file_list = [f for f in os.listdir(folder_path)]

df_dict = {}
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    key = file_name
    df_dict[key] = pd.read_excel(
        file_path, sheet_name='Final Scores', usecols="A:C", header=2
    )

# Create dataframe for podium
main_podium = pd.DataFrame()
# Join the files
for key in df_dict:
    podium = df_dict[key].rename(columns={df_dict[key].columns[0]: 'Podium'})
    main_podium = pd.concat([main_podium, podium])

# Rename columns
main_podium.rename(columns={'Total Score (points)': 'Points'}, inplace=True)
# Change data type
main_podium['Podium'] = main_podium['Podium'].astype(int)
main_podium['Points'] = main_podium['Points'].astype(int)

# Clean names
main_podium['Player'] = main_podium['Player'].apply(
    lambda x: clean_string(x))

# Substitute name aliases
main_podium['Player'] = main_podium['Player'].replace(name_alias)

# Assign Podium points
point_mapping = {1: 3, 2: 2, 3: 1}
main_podium['Podium_Points'] = main_podium['Podium'].map(point_mapping)

In [45]:
# [DEBUG] Display the unique players to verify
logging.debug(f'Unique Players raw:\n{sorted(main_podium['Player'].unique())}')

2025-04-28 10:45:39 [DEBUG] Unique Players raw:
['', 'allan', 'ange', 'anna', 'bidomar', 'brenda', 'bruna', 'byby', 'cyberlambari', 'douglas', 'edy', 'erick', 'evelyn', 'fazol', 'feliznatal', 'feriza', 'fernando', 'flaviao', 'flavio', 'gabe', 'gabriel', 'gustavo', 'jd', 'juniro', 'lambari', 'lambarirobo', 'le', 'lilrodi', 'loren', 'luara', 'marbi', 'marcia', 'mardabi', 'mari', 'mariana', 'marilyna', 'matusalem', 'moicano', 'natal', 'natalmatheus', 'nath', 'oliver', 'panduca', 'pmd', 'rafa', 'rederd', 'robozinlambari', 'tety', 'thais', 'thoomas', 'tobias', 'trentin', 'veruska', 'xandao', 'yejn']


## Final ranking

In [46]:
# Create final ranking
rank = (main_podium.loc[:, ['Player', 'Podium_Points', 'Points']]
        .groupby(['Player'])
        .sum()
        .reset_index())
# Index starts at 1
rank.index = rank.index + 1

In [47]:
# [DEBUG] Display the unique players to verify
logging.debug(f'Unique Players:\n{sorted(main_podium['Player'].unique())}')

2025-04-28 10:45:39 [DEBUG] Unique Players:
['', 'allan', 'ange', 'anna', 'bidomar', 'brenda', 'bruna', 'byby', 'cyberlambari', 'douglas', 'edy', 'erick', 'evelyn', 'fazol', 'feliznatal', 'feriza', 'fernando', 'flaviao', 'flavio', 'gabe', 'gabriel', 'gustavo', 'jd', 'juniro', 'lambari', 'lambarirobo', 'le', 'lilrodi', 'loren', 'luara', 'marbi', 'marcia', 'mardabi', 'mari', 'mariana', 'marilyna', 'matusalem', 'moicano', 'natal', 'natalmatheus', 'nath', 'oliver', 'panduca', 'pmd', 'rafa', 'rederd', 'robozinlambari', 'tety', 'thais', 'thoomas', 'tobias', 'trentin', 'veruska', 'xandao', 'yejn']


## Podium Variations

In [48]:
logging.info('Ranking por Podium Points e desempate por Points\n')

rank.sort_values(['Podium_Points', 'Points'],ascending=[False, False]) \
    .head(5) \
    .reset_index(drop=True)

2025-04-28 10:45:39 [INFO] Ranking por Podium Points e desempate por Points



,Player,Podium_Points,Points
0,brenda,14.0,42482
1,luara,9.0,23526
2,feriza,8.0,25494
3,natal,3.0,15639
4,marbi,3.0,9725


In [54]:
# TOP 5 PODIUM PTS AND THEN BY ACCUMULATED PTS
logging.info('\nRanking por top 5 Points e ordenado por Podium Points')

print(folder_path)

rank.sort_values(['Points'], ascending=False) \
    .head(5) \
    .sort_values(['Podium_Points'], ascending=False) \
    .reset_index(drop=True)

2025-04-28 10:48:49 [INFO] 
Ranking por top 5 Points e ordenado por Podium Points


2025-1_METANOIA


,Player,Podium_Points,Points
0,brenda,14.0,42482
1,luara,9.0,23526
2,feriza,8.0,25494
3,pmd,1.0,22766
4,trentin,1.0,20088


In [50]:
logging.info('Ranking por Points')

rank.sort_values(['Points'], ascending=False) \
    .head(5) \
    .reset_index(drop=True)

2025-04-28 10:45:39 [INFO] Ranking por Points


,Player,Podium_Points,Points
0,brenda,14.0,42482
1,feriza,8.0,25494
2,luara,9.0,23526
3,pmd,1.0,22766
4,trentin,1.0,20088


In [51]:
file_list

['METANOIA [02] Restaurado por completo.xlsx',
 'METANOIA [03] A mente de Cristo.xlsx',
 'METANOIA [06] Resistindo à tentação.xlsx',
 'METANOIA [07] Acalmando a ira.xlsx',
 'METANOIA [08] Experimente o perdão.xlsx',
 'METANOIA [09] Encontrar paz interior.xlsx',
 'METANOIA [11] Cura no Sofrimento.xlsx',
 'METANOIA [12] Sentindo-se Realizado.xlsx',
 'METANOIA [13] Resgatando os frágeis.xlsx']

In [52]:
# [DEBUG] Assiduidade | Presença
main_podium.groupby('Player')['Points'] \
           .count() \
           .sort_values(ascending=False) \
           .reset_index() \
           .rename(columns={'Points': 'Presença'})

,Player,Presença
0,brenda,9
1,evelyn,7
2,luara,6
3,pmd,6
4,trentin,5
5,feriza,5
6,ange,4
7,natal,4
8,natalmatheus,4
9,veruska,4
